# Plan
* Compare naive aggregate stats to the model
  * Present the final form of the model again
  * do assumptions check on this
  * consider the impact of slashed validators on this figure
* Do a more detailed breakdown of aggregate stats (look at inclusion, target, head, delay individually)
* Do this on a per validator basis - allow any validator to see exactly which types of failure caused them to fall short

# Things to look at
* proportion of validators with effective balance below 32 ETH
* distribution of validator incomes
* comparison of average validator reward with the expected case from the model
* per-validator breakdown explaining why they fell short of the maximum possible (missed attestations, missed proposals, incorrect target/head, delayed inclusion)
* per-validator stats (longest outage, longest streak, uptime, proposal opportunities (how lucky?), proportion of successful proposals, inclusion delays (delay due to missed block versus not included in block - how lucky? how many validators have not produced a single attestation or appear to have been abandoned?)
* Does the distribution of block proposal opportunities match the binomial as expected?

# To do
* add a date x-axis to all charts with epoch or slot numbers

In [1]:
# imports, constants

import time

import psycopg2
import matplotlib.pyplot as plt
import pandas as pd

FAR_FUTURE_EPOCH = 2**64 - 1 # as defined in spec

In [45]:
# open/restart connection to chaind database

try:
    cursor.close()
    connection.close()
except:
    pass

connection = psycopg2.connect(user="chain", host="127.0.0.1", database="chain", password="medalla")
cursor = connection.cursor()

In [3]:
# define progress indicator function

def print_progress(start_time, current_item, n_items):
    seconds = time.time() - start_time
    elapsed = time.strftime("%H:%M:%S",time.gmtime(seconds))
    left = time.strftime("%H:%M:%S",time.gmtime(seconds * ((n_items) / (current_item+1)-1)))
    perc = 100*(current_item+1)/n_items
    print(f"iteration {current_item} of {n_items} ({perc:.2f}%) / {elapsed} elapsed / {left} left", end='\r')

In [37]:
# get validator and epoch summary data

cursor.execute("SELECT f_activation_eligibility_epoch, f_activation_epoch, f_exit_epoch, "
               "f_slashed, f_public_key FROM t_validators ORDER BY f_index")
result = cursor.fetchall()
validators = [{
    "index"                       : i,
    "activation_eligibility_epoch": r[0],
    "activation_epoch"            : r[1],
    "exit_epoch"                  : r[2],
    "slashed"                     : r[3],
    "slashed_slot"                : None,
    "slashed_type"                : None,
    "pubkey"                      : r[4].hex()
} for i, r in enumerate(result)]

pubkey_lookup = {r[4].hex(): validators[i] for i, r in enumerate(result)}

for v in validators:
    if v["activation_epoch"] is None:
        v["activation_epoch"] = FAR_FUTURE_EPOCH
    if v["exit_epoch"] is None:
        v["exit_epoch"] = FAR_FUTURE_EPOCH

cursor.execute("SELECT * FROM t_epoch_summaries WHERE f_epoch < 32002 ORDER BY f_epoch")
result = cursor.fetchall()
epoch_summaries = [{
    "epoch"                           : r[0],
    "activation_queue_length"         : r[1],
    "activating_validators"           : r[2],
    "active_validators"               : r[3],
    "active_real_balance"             : r[4],
    "active_balance"                  : r[5],
    "attesting_validators"            : r[6],
    "attesting_balance"               : r[7],
    "target_correct_validators"       : r[8],
    "target_correct_balance"          : r[9],
    "head_correct_validators"         : r[10],
    "head_correct_balance"            : r[11],
    "attestations_for_epoch"          : r[12],
    "attestations_in_epoch"           : r[13],
    "duplicate_attestations_for_epoch": r[14],
    "proposer_slashings"              : r[15],
    "attester_slashings"              : r[16],
    "deposits"                        : r[17],
    "exiting_validators"              : r[18],
    "canonical_blocks"                : r[19]
} for r in result]

In [38]:
# check for any instances of validator deposits made to already active validators

cursor.execute("SELECT f_inclusion_slot, f_validator_pubkey, f_amount FROM t_deposits")
deposits = cursor.fetchall()

repeat_deposit_count = 0
repeat_deposit_epochs = {}
for deposit in deposits:
    slot, tmp, amount = deposit
    pubkey = tmp.hex()
    if pubkey not in pubkey_lookup:
        continue
    else:
        validator = pubkey_lookup[deposit[1].hex()]
    if slot // 32 > validator["activation_epoch"]:
        repeat_deposit_count += 1
        epoch = (slot - 1) // 32 + 1 # NB chaind balances are based on the state *after* processing slot 0
        #print(f"validator {validator['index']} made duplicate deposit of {amount} in epoch {epoch}")
        cursor.execute(f"SELECT f_epoch, f_balance FROM t_validator_balances "
                       f"WHERE f_epoch BETWEEN {epoch-1} AND {epoch+1} "
                       f"AND f_validator_index = {validator['index']}")
        balances = cursor.fetchall()
        #for row in balances:
        #    print(f"epoch: {row[0]}, balance: {row[1]}")
            
        if epoch in repeat_deposit_epochs:
            repeat_deposit_epochs[epoch] += deposit[2]
        else:
            repeat_deposit_epochs[epoch] = deposit[2]

print(f"{repeat_deposit_count} repeat deposits found")    

5 repeat deposits found


In [32]:
# create a dict of slashing events, identifying slashed validator, inclusion slot and slashing type

cursor.execute("SELECT f_inclusion_slot, f_header_1_proposer_index FROM t_proposer_slashings")
proposer_slashings = cursor.fetchall()

combined = []

for slashing in proposer_slashings:
    validators[slashing[1]]["slashed_slot"] = slashing[0]
    validators[slashing[1]]["slashed_type"] = "proposer"
    combined += [[slashing[0], slashing[1], "proposer"]]

cursor.execute("SELECT f_inclusion_slot, f_attestation_1_indices, f_attestation_2_indices "
               " FROM t_attester_slashings")
attester_slashings = cursor.fetchall()
for slashing in attester_slashings:
    for index in slashing[1]:
        if index in slashing[2]:
            validators[index]["slashed_slot"] = slashing[0]
            validators[index]["slashed_type"] = "attester"
            combined += [[slashing[0], index, "attester"]]

print(f"{len(combined)} slashing events found")
#l = sorted(combined, key = lambda k: k[0])
#for s in combined:
#    print(s)

135 slashing events found


In [43]:
for event in combined:
    epoch = event[0] // 32
    cursor.execute(f"SELECT f_balance FROM t_validator_balances "
                   f"WHERE f_validator_index = {event[1]} AND f_epoch BETWEEN {epoch-2} AND {epoch+2}")
    balances = [b[0] for b in cursor.fetchall()]
    print(f"validator {event[1]} {event[2]} violation at slot {event[0]} (epoch {epoch}.{event[0] % 32})")
    for i, balance in enumerate(balances):
        print(f"epoch {epoch-2+i}: {balance}")

validator 20075 proposer violation at slot 6669 (epoch 208.13)
epoch 206: 32014788155
epoch 207: 32014860472
epoch 208: 32014933849
epoch 209: 31764875586
epoch 210: 31766407014
validator 18177 proposer violation at slot 22374 (epoch 699.6)
epoch 697: 32049805699
epoch 698: 32049876895
epoch 699: 32049948103
epoch 700: 31799892297
epoch 701: 31801547694
validator 25645 proposer violation at slot 40772 (epoch 1274.4)
epoch 1272: 32005449786
epoch 1273: 32005517920
epoch 1274: 32005586091
epoch 1275: 31755532811
epoch 1276: 31757261394
validator 38069 proposer violation at slot 138164 (epoch 4317.20)
epoch 4315: 32000913350
epoch 4316: 32000969250
epoch 4317: 32001025538
epoch 4318: 31750981570
epoch 4319: 31753109171
validator 38089 proposer violation at slot 138731 (epoch 4335.11)
epoch 4333: 32001741954
epoch 4334: 32001798291
epoch 4335: 32001854532
epoch 4336: 31751819753
epoch 4337: 31753937057
validator 38130 proposer violation at slot 140313 (epoch 4384.25)
epoch 4382: 3200357019

In [ ]:
# calculate aggregate net rewards (from the change in the balances of active validators)

cursor.execute("SELECT f_balance FROM t_validator_balances WHERE f_epoch = 1 ORDER BY f_validator_index")
prior_balance = cursor.fetchall()
start_time = time.time()
for e, s in enumerate(epoch_summaries):
    if e+2 >= len(epoch_summaries):
        break
    s["aggregate_net_reward"] = 0
    cursor.execute(f"SELECT f_balance FROM t_validator_balances "
                   f"WHERE f_epoch = {e+2} ORDER BY f_validator_index")
    new_balances = cursor.fetchall()
    for validator_index, balance in enumerate(new_balances):
        validator = validators[validator_index]
        if validator
        if validator["activation_epoch"] <= e and (validator["exit_epoch"] > e):
            s["aggregate_net_reward"] += balance[0] - prior_balances[validator_index][0]
    
    prior_balances = new_balances
    print_progress(start_time, e, len(epoch_summaries) - 2)

In [ ]:
# store aggregate net rewards in chaind database

for s in epoch_summaries:
    if "aggregate_net_reward" in s:
        e = s["epoch"]
        anr = s["aggregate_net_reward"]
        cursor.execute(f"INSERT INTO t_epoch_extras (f_epoch, f_aggregate_net_reward) VALUES ({e}, {anr})")

connection.commit()

In [ ]:
# count waiting validators

not_activated_count = 0
for v in validators:
    if v['activation_epoch'] is None:
        not_activated_count += 1
        
not_activated_count

In [47]:
# get validator status by epoch - waiting, active, exited, slashed
start_time = time.time()

waiting_count = [0] * n_epochs
active_count  = [0] * n_epochs
slashed_count = [0] * n_epochs
exited_count  = [0] * n_epochs

for epoch in range(n_epochs):
    for validator in validators:
        activation_epoch = n_epochs if validator['activation_epoch'] is None else validator['activation_epoch']
        if epoch >= validator['activation_eligibility_epoch'] and epoch < activation_epoch:
            waiting_count[epoch] += 1
        elif validator['exit_epoch'] and epoch >= validator['exit_epoch'] :
            if validator['slashed']:
                slashed_count[epoch] += 1
            else:
                exited_count[epoch] += 1
        elif epoch >= activation_epoch:
            active_count[epoch] += 1

    print_progress(start_time, epoch, n_epochs)

print(f"completed in {elapsed}" + ' ' * 50)       

status_data = pd.DataFrame({'waiting': waiting_count, 'active': active_count,
                            'exited': exited_count, 'slashed': slashed_count})

NameError: name 'n_epochs' is not defined

In [46]:
# draw validator status stacked area plot
fig=plt.figure(figsize=(12,8))
#plt.stackplot(range(len(status_data.index)), status_data['active'], status_data['slashed'],
#              status_data['exited'], status_data['waiting'],labels=['active','slashed','exited','waiting'],
#              colors=['tab:blue','tab:red','tab:green','tab:orange'])
plt.stackplot(range(2000), status_data['active'].iloc[0:2000], status_data['slashed'].iloc[0:2000],
              status_data['exited'].iloc[0:2000], status_data['waiting'].iloc[0:2000],labels=['active','slashed','exited','waiting'],
              colors=['tab:blue','tab:red','tab:green','tab:orange'])
plt.legend(loc='right')
plt.margins(0,0)
plt.title('Validator status')
plt.xlabel('epoch')
plt.ylabel('number of validators')
plt.show()

NameError: name 'status_data' is not defined

<Figure size 864x576 with 0 Axes>

In [ ]:
# show effective balances

cursor.execute(f"SELECT f_effective_balance FROM t_validator_balances WHERE f_validator_index IN"
               f"(SELECT f_index FROM t_validators WHERE NOT f_slashed  AND f_activation_epoch != -1)"
               f"AND f_epoch = {n_epochs-1}")
eff_balances = pd.Series([b[0]*10**-9 for b in cursor.fetchall()], name='non-slashed')

cursor.execute(f"SELECT f_effective_balance FROM t_validator_balances WHERE f_validator_index IN"
               f"(SELECT f_index FROM t_validators WHERE f_slashed)"
               f"AND f_epoch = {n_epochs-1}")
eff_balances_slashed = pd.Series([b[0]*10**-9 for b in cursor.fetchall()], name='slashed')

print(f"Non-slashed validator balance (min, max): "
      f"({int(eff_balances.min())}, {int(eff_balances.max())}) ETH")
print(f"Slashed validator balance (min, max): "
      f"({int(eff_balances_slashed.min())}, {int(eff_balances_slashed.max())}) ETH")

cursor.execute(f"SELECT COUNT(*) FROM t_validator_balances WHERE f_validator_index IN "
               f"(SELECT f_index FROM t_validators WHERE NOT f_slashed AND f_activation_epoch != -1 "
               f"AND f_effective_balance < 32000000000) AND f_epoch = {n_epochs-1}")
n_reduced_balance = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM t_validators WHERE NOT f_slashed AND f_activation_epoch != -1")
n_not_slashed = cursor.fetchone()[0]

print(f"{n_reduced_balance} non-slashed validators have an effective balance of less than 32 ETH "
      f"({100 * n_reduced_balance / n_not_slashed:.2f}% of non-slashed validators)")